In [1]:
using GridapGmsh
using Gmsh: gmsh
using Gridap
using Gridap.Geometry
using Gridap.TensorValues
using LineSearches: BackTracking
using LinearAlgebra

In [2]:
const E_mat = 36.2e3
const ν_mat = 0.21

const G₁₂_mat = E_mat/(2*(1+ν_mat))

const λ_mat = E_mat*ν_mat/((1+ν_mat)*(1-2*ν_mat))
const μ_mat = E_mat/(2*(1+ν_mat))
const k_mat = λ_mat + μ_mat

25790.82359646623

In [3]:
const α = 10
const GcI = 0.011
const GcII = α * GcI

const β = 2
const σcI = 10
const σcII = β * σcI

20

In [4]:
const η = 1e-15

1.0e-15

In [5]:
const ls = 1.0

1.0

In [6]:
mI(GcIc) = 4*GcIc*k_mat/(pi*ls*σcI^2)
mII(GcIIc) = 4*GcIIc*μ_mat/(pi*ls*σcII^2)

const ψ_Crit_I = σcI^2/(2*k_mat)
const ψ_Crit_II = σcII^2/(2*μ_mat)

0.013370165745856354

In [7]:
const L = 102
const H = 102
const a = 20.32*0.5
const θ = pi/4

const CH = 0.204
const hfc = ls/2.1
const hf = ls/2.1
const h = 10*hf 
const FMR = 5*0.5

#Coordinates
A1 = 0.5*L - a*cos(θ) + CH*sin(θ)
A2 = 0.5*H - a*sin(θ) - CH*cos(θ)

B1 = 0.5*L - a*cos(θ) - CH*sin(θ)
B2 = 0.5*H - a*sin(θ) + CH*cos(θ)

C1 = 0.5*L + a*cos(θ) - CH*sin(θ)
C2 = 0.5*H + a*sin(θ) + CH*cos(θ)

D1 = 0.5*L + a*cos(θ) + CH*sin(θ)
D2 = 0.5*H + a*sin(θ) - CH*cos(θ)

gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.geo.addPoint(0.0, 0.0, 0.0, h, 1)  
gmsh.model.geo.addPoint(L, 0.0, 0.0, h, 2) 
gmsh.model.geo.addPoint(L, H, 0.0, h, 3) 
gmsh.model.geo.addPoint(0.0, H, 0.0, h, 4)

gmsh.model.geo.addPoint(A1, A2, 0.0, h, 5)
gmsh.model.geo.addPoint(B1, B2, 0.0, h, 6)
gmsh.model.geo.addPoint(C1, C2, 0.0, h, 7)
gmsh.model.geo.addPoint(D1, D2, 0.0, h, 8)

gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(2, 3, 2)
gmsh.model.geo.addLine(3, 4, 3)
gmsh.model.geo.addLine(4, 1, 4)

gmsh.model.geo.addLine(5, 6, 5)
gmsh.model.geo.addLine(6, 7, 6)
gmsh.model.geo.addLine(7, 8, 7)
gmsh.model.geo.addLine(8, 5, 8)

gmsh.model.geo.addLine(1, 3, 55)

gmsh.model.geo.addCurveLoop([1,2,3,4],1) 
gmsh.model.geo.addCurveLoop([5,6,7,8],2) 

gmsh.model.geo.addPlaneSurface([1,-2], 1)

gmsh.model.addPhysicalGroup(2, [1],1)

gmsh.model.addPhysicalGroup(1, [1],1)
gmsh.model.addPhysicalGroup(1, [3],2)

gmsh.model.setPhysicalName(2, 1, "Domain")
gmsh.model.setPhysicalName(1, 1, "DirichletBot")
gmsh.model.setPhysicalName(1, 2, "DirichletTop")


gmsh.model.mesh.field.add("Distance", 1)
gmsh.model.mesh.field.setNumbers(1, "EdgesList", [55])

gmsh.model.mesh.field.add("Threshold", 2)
gmsh.model.mesh.field.setNumber(2, "IField", 1)
gmsh.model.mesh.field.setNumber(2, "LcMin", hf)
gmsh.model.mesh.field.setNumber(2, "LcMax", h)
gmsh.model.mesh.field.setNumber(2, "DistMin", 30*FMR)
gmsh.model.mesh.field.setNumber(2, "DistMax", 32*FMR)

gmsh.model.mesh.field.setAsBackgroundMesh(2)

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)
gmsh.write("SquarePlateWithEdgeNotch.msh")
gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 30%] Meshing curve 3 (Line)
Info    : [ 40%] Meshing curve 4 (Line)
Info    : [ 50%] Meshing curve 5 (Line)
Info    : [ 60%] Meshing curve 6 (Line)
Info    : [ 70%] Meshing curve 7 (Line)
Info    : [ 80%] Meshing curve 8 (Line)
Info    : [ 90%] Meshing curve 55 (Line)
Info    : Done meshing 1D (Wall 0.000856282s, CPU 0.000857s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 1.83492s, CPU 1.81817s)
Info    : 57411 nodes 114529 elements
Info    : Writing 'SquarePlateWithEdgeNotch.msh'...
Info    : Done writing 'SquarePlateWithEdgeNotch.msh'


In [8]:
model = GmshDiscreteModel("SquarePlateWithEdgeNotch.msh")
writevtk(model,"SquarePlateWithEdgeNotch")  

Info    : Reading 'SquarePlateWithEdgeNotch.msh'...
Info    : 18 entities
Info    : 57109 nodes
Info    : 113700 elements
Info    : Done reading 'SquarePlateWithEdgeNotch.msh'


3-element Vector{Vector{String}}:
 ["SquarePlateWithEdgeNotch_0.vtu"]
 ["SquarePlateWithEdgeNotch_1.vtu"]
 ["SquarePlateWithEdgeNotch_2.vtu"]

In [9]:
I2 = SymTensorValue{2,Float64}(1.0 ,0.0 ,1.0)
I4 = I2⊗I2
I4_sym = one(SymFourthOrderTensorValue{2,Float64})
I4_vol = (1.0/2)*I4
I4_dev = I4_sym  - I4_vol  

SymFourthOrderTensorValue{2, Float64, 9}(0.5, 0.0, -0.5, 0.0, 0.5, 0.0, -0.5, 0.0, 0.5)

In [10]:
p = -0.5

-0.5

In [11]:
function degDer1(ϕ,GcI)
    mIc = mI(GcI)
    g = ((ϕ-1)*(ϕ*(2*p+1)+1)*mIc)/((ϕ*ϕ*(mIc*p+1)+(ϕ*(mIc-2)+1))^2) 
    return g
end

degDer1 (generic function with 1 method)

In [12]:
function degDer2(ϕ,GcII)
    mIIc = mII(GcII)
    g = ((ϕ-1)*(ϕ*(2*p+1)+1)*mIIc)/((ϕ*ϕ*(mIIc*p+1)+(ϕ*(mIIc-2)+1))^2) 
    return g
end

degDer2 (generic function with 1 method)

In [13]:
function σfun(ε,ε_in,s_in,GcI,GcII)
    mIc = mI(GcI)
    mIIc = mII(GcII)
    εdev = I4_dev ⊙ ε
    σI = k_mat*tr(ε)*one(ε)
    σII = 2*μ_mat*εdev
    σ = ((1-s_in)^2 / ((1-s_in)^2 + ((mIc*s_in*(1 + p*s_in)))))*(σI) + ((1-s_in)^2 / ((1-s_in)^2 + ((mIIc*s_in*(1 + p*s_in)))))*(σII)
    return  σ
end  

σfun (generic function with 1 method)

In [14]:
function Eigen(ε)
    εArray = get_array(ε)
    Λ, P = eigen(εArray)
    ε1 = Λ[1]
    ε2 = Λ[2]
    if ε1 >= 0 &&  ε2 >= 0
        Λpos = [ε1 0; 0 ε2]
        Λneg = [0 0; 0 0]
    elseif ε1 >= 0 &&  ε2 < 0
        Λpos = [ε1 0; 0 0]
        Λneg = [0 0; 0 ε2]
    elseif ε1 < 0 &&  ε2 >= 0
        Λpos = [0 0; 0 ε2]
        Λneg = [ε1 0; 0 0]
    elseif ε1 < 0 &&  ε2 < 0
        Λpos = [0 0; 0 0]
        Λneg = [ε1 0; 0 ε2]
    end
    εPos = TensorValue(P*Λpos*P')
    εNeg = TensorValue(P*Λneg*P')
    return εPos, εNeg
end

Eigen (generic function with 1 method)

In [15]:
function ψPosTen(ε_in)
    εdev = I4_dev ⊙ ε_in
    εPos, εNeg = Eigen(εdev) 
    if tr(ε_in) > 0.0
        ψPlus = 0.5*k_mat*(tr(ε_in))^2
    else
        ψPlus = 0.0
    end
    return ψPlus
end

ψPosTen (generic function with 1 method)

In [16]:
function ψPosShear(ε_in)
    εdev = I4_dev ⊙ ε_in
    εPos, εNeg = Eigen(εdev) 
    ψPlus = μ_mat * (εPos ⊙ εPos)
    return ψPlus
end

ψPosShear (generic function with 1 method)

In [17]:
function new_EnergyState(ψPlusPrev_in,ψhPos_in)
  ψPlus_in = ψhPos_in
  if ψPlus_in >= ψPlusPrev_in
    ψPlus_out = ψPlus_in
  else
    ψPlus_out = ψPlusPrev_in
  end
  true,ψPlus_out
end

new_EnergyState (generic function with 1 method)

In [18]:
function EnergyState(ψPlusPrev_in,ψhPos_in)
  ψPlus_out = ψhPos_in
  true,ψPlus_out
end

EnergyState (generic function with 1 method)

In [19]:
function  project(q,model ,dΩ,order)
    reffe = ReferenceFE(lagrangian ,Float64 ,order)
    V = FESpace(model ,reffe ,conformity =:L2)
    a(u,v) =∫(u*v)*dΩ
    b(v) =∫(v*q)*dΩ
    op = AffineFEOperator(a,b,V,V)
    qh = Gridap.solve(op)
    return  qh
end
     

project (generic function with 1 method)

In [20]:
order = 1
reffe_PF = ReferenceFE(lagrangian ,Float64,order)
V0_PF = TestFESpace(model ,reffe_PF;conformity =:H1)
U_PF = TrialFESpace(V0_PF)
sh = zero(V0_PF)

SingleFieldFEFunction():
 num_cells: 113270
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 17104460867162936782

In [21]:
reffe_Disp = ReferenceFE(lagrangian,VectorValue{2,Float64},order)
        V0_Disp = TestFESpace(model,reffe_Disp;
          conformity=:H1,
          dirichlet_tags=["DirichletBot","DirichletTop"],
          dirichlet_masks=[(true,true), (false,true)])

uh = zero(V0_Disp)

SingleFieldFEFunction():
 num_cells: 113270
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 17104460867162936782

In [22]:
degree = 2*order
Ω= Triangulation(model)
dΩ= Measure(Ω,degree) 

GenericMeasure()

In [23]:
labels = get_face_labeling(model)
LoadTagId = get_tag_from_name(labels,"DirichletTop")
Γ_Load = BoundaryTriangulation(model,tags = LoadTagId)
dΓ_Load = Measure(Γ_Load,degree)
n_Γ_Load = get_normal_vector(Γ_Load)

GenericCellField():
 num_cells: 215
 DomainStyle: ReferenceDomain()
 Triangulation: BoundaryTriangulation()
 Triangulation id: 1945304062373542040

In [24]:
nls = NLSolver(
  show_trace=true,
  method=:newton,
  linesearch=BackTracking(), iterations = 10)
solver = FESolver(nls)

NonlinearFESolver()

In [25]:
function run_PF(x0,vApp,cache,GcI,GcII,ψTenPrev_in,ψShearPrev_in)
     U_PF = TrialFESpace(V0_PF)
     res(s,ϕ) = ∫((2/pi)*ls*∇(ϕ) ⋅ ∇(s) + (degDer1∘(s,GcI))*(ψTenPrev_in/GcI)*ϕ + (degDer2∘(s,GcII))*(ψShearPrev_in/GcII)*ϕ + ((1/pi)*(2-2*s)/ls)*ϕ)*dΩ
     op = FEOperator(res,U_PF,V0_PF)
     sh_out = FEFunction(U_PF,x0)
     sh_out, cache = solve!(sh_out,solver,op,cache)
  return sh_out, get_free_dof_values(sh_out), cache
end

run_PF (generic function with 1 method)

In [26]:
function  stepDisp(uh_in,sh_in,vApp,GcI,GcII)
    uApp1(x) = VectorValue(0.0,0.0)
    uApp2(x) = VectorValue(0.0,-vApp)
    U_Disp = TrialFESpace(V0_Disp,[uApp1 ,uApp2])
    a_Disp(u,v) =∫( (ε(v)⊙(σfun∘(ε(u),ε(uh_in),sh_in,GcI,GcII)) ) )*dΩ
    b_Disp(v) = 0.0
    op_Disp = AffineFEOperator(a_Disp ,b_Disp ,U_Disp ,V0_Disp)
    uh_out = solve(op_Disp)
    return  uh_out
end

stepDisp (generic function with 1 method)

In [27]:
function χFun1(ψhTenPos_in,ψhShearPos_in)
    χVal = (ψhTenPos_in/(ψhShearPos_in + ψhTenPos_in))^2
    return χVal
end

function χFun2(ψhTenPos_in,ψhShearPos_in)
    χVal = (ψhShearPos_in/(ψhShearPos_in + ψhTenPos_in))^2
    return χVal
end

χFun2 (generic function with 1 method)

In [28]:
function GcFunI(χVal1)
    Gc = GcII + (GcI - GcII)*χVal1
    return Gc
end

function GcFunII(χVal2)
    Gc = GcI + (GcII - GcI)*χVal2
    return Gc
end

GcFunII (generic function with 1 method)

In [29]:
function ψCritFun(χVal1,χVal2)
    ψCrit_I_in = χVal1 * ψ_Crit_I
    ψCrit_II_in = χVal2 * ψ_Crit_II
    return ψCrit_I_in,ψCrit_II_in
end

ψCritFun (generic function with 1 method)

In [ ]:
vApp = 0.0
const vAppMax = 0.10
delv = 0.05/100 #1e-4
innerMax = 10
count = 0
Load = Float64[]
Displacement = Float64[]
push!(Load, 0.0)
push!(Displacement, 0.0)

χValPrev1 = 0.5 
χValPrev2 = 0.5 

χVal1 = CellState(χValPrev1,dΩ)
χVal2 = CellState(χValPrev2,dΩ)

GcPrevI = GcII + (GcI - GcII)*χValPrev1
GcPrevII = GcI + (GcII - GcI)*χValPrev2

ψCrit_I_Prev = (χValPrev1)*ψ_Crit_I
ψCrit_II_Prev = (χValPrev2)*ψ_Crit_II

GcIV = project(CellState(GcPrevI,dΩ) ,model ,dΩ,order) 
GcIIV = project(CellState(GcPrevII,dΩ) ,model ,dΩ,order) 

ψTenPrev = CellState(ψCrit_I_Prev,dΩ)
ψShearPrev = CellState(ψCrit_II_Prev,dΩ)

x0_PF = zeros(Float64,num_free_dofs(V0_PF))
cache_1 = nothing

uh = zero(V0_Disp)
sh = zero(V0_PF)

sh_Prev = sh
uh_Prev = uh

while  vApp .< vAppMax
    count = count  .+ 1
    
    #=if vApp <= 4e-3
        delv = 2e-4
    else
        delv = 1e-5
end=#
    
    vApp = vApp .+ delv
    
    print("\n Entering  displacemtent  step$count :", float(vApp))
        
    for  inner = 1: innerMax
        
        ψhTenPrev = project(ψTenPrev ,model ,dΩ,order)
        ψhShearPrev = project(ψShearPrev ,model ,dΩ,order)
        
        RelErr = abs(sum(∫( (2/pi)*ls*∇(sh)⋅ ∇(sh) + (degDer1∘(sh,GcIV))*(ψhTenPrev/GcIV)*sh + (degDer2∘(sh,GcIIV))*(ψhShearPrev/GcIIV)*sh - (2/(pi*ls))*sh*sh)*dΩ + ∫( (2/(pi*ls))*sh)*dΩ))/abs(sum(∫( (2/(pi*ls))*sh)*dΩ))
 
        #RelErr = max(RelErr1,RelErr2)
        
        println("\nRelative error =", float(RelErr), "\n")
    
        sh,x0_PF,cache_1 = run_PF(x0_PF,vApp,cache_1,GcIV,GcIIV,ψhTenPrev,ψhShearPrev)
        uh = stepDisp(uh,sh,vApp,GcIV,GcIIV)
        
        ψhTenPos_in =ψPosTen∘(ε(uh))
        ψhShearPos_in =ψPosShear∘(ε(uh))
        
        update_state!(new_EnergyState ,ψTenPrev ,ψhTenPos_in)
        update_state!(new_EnergyState ,ψShearPrev ,ψhShearPos_in)
        
        χVal1 = χFun1∘(ψTenPrev,ψShearPrev)
        χVal2 = χFun2∘(ψTenPrev,ψShearPrev)
        
        GcIV = GcFunI(χVal1)
        GcIIV = GcFunII(χVal2)
        ψCrit_I_in,ψCrit_II_in = ψCritFun(χVal1,χVal2)
        
        
        update_state!(new_EnergyState ,ψTenPrev, ψCrit_I_in)
        update_state!(new_EnergyState ,ψShearPrev, ψCrit_II_in)
        
        #ψTenPrev = ψCrit_ISt
        #ψShearPrev = ψCrit_IISt

        
        if   RelErr  < 1e-4 || isnan(RelErr)
            break
        end
        
    end
    GcVal1 = project(GcIV ,model ,dΩ,order)
    GcVal2 = project(GcIIV ,model ,dΩ,order)
    
    χValC1 = project(χVal1 ,model ,dΩ,order)
    χValC2 = project(χVal2 ,model ,dΩ,order)
    
    Node_Force = sum(∫(n_Γ_Load⋅(σfun∘(ε(uh),ε(uh),sh,GcVal1,GcVal2)))*dΓ_Load)
    
    push!(Load , -Node_Force[2])
    push!( Displacement , vApp)
    if mod(count,2) == 0
         writevtk(Ω,"Modified_BKCriteria_Plate_With_Inclined_Central_Crack$count",cellfields=
        ["uh"=>uh,"s"=>sh , "epsi"=>ε(uh), "modefactorI"=>χValC1, "modefactorII"=>χValC2])
    end
    
end


 Entering  displacemtent  step1 :0.0005
Relative error =NaN

Iter     f(x) inf-norm    Step 2-norm 
------   --------------   --------------
     0     2.188701e-17              NaN

 Entering  displacemtent  step2 :0.001
Relative error =NaN

Iter     f(x) inf-norm    Step 2-norm 
------   --------------   --------------
     0     4.622707e-02              NaN
     1     1.149469e-02     2.747312e+00
     2     1.143741e-03     5.348641e-01
     3     1.374713e-05     8.051019e-03
     4     2.033081e-09     1.220740e-06

 Entering  displacemtent  step3 :0.0015
Relative error =0.06591340899050976

Iter     f(x) inf-norm    Step 2-norm 
------   --------------   --------------
     0     1.159598e-02              NaN
     1     1.224393e-03     8.227125e-01
     2     1.657131e-05     1.431117e-02
     3     3.112095e-09     2.768259e-06


In [ ]:
using Plots

In [ ]:
plot(Displacement,Load)
#xlims!(0,0.006)

In [ ]:
maximum(Load)

In [ ]:
using DelimitedFiles
using CSV

In [ ]:
using LaTeXStrings

In [ ]:
open("load_dispPFCZMNew$ls.csv", "w") do io
             writedlm(io, [Displacement Load])
end